In [ ]:
import pandas as pd
from config import username, password
from sqlalchemy import create_engine

## Import Google csv with selective columns

In [ ]:
csv_path = r'Resources/googleplaystore.csv'
google_columns = ['App', 'Category', 'Rating', 'Reviews', 'Price']
google_df = pd.read_csv(csv_path, usecols=google_columns)

# to avoid issues with inserting datat into pg, all column names are lowercase
google_df.columns = [col.lower() for col in google_columns]

# remove $ from price to align formatting
google_df.price = google_df.price.str.replace('$', '', regex=False)

# convert rating, reviews, and price to numeric type
google_df.rating = pd.to_numeric(google_df.rating, errors='coerce')
google_df.reviews = pd.to_numeric(google_df.reviews, errors='coerce')
google_df.price = pd.to_numeric(google_df.price, errors='coerce')

# drop na rows
google_df.dropna(inplace=True)

google_df.head()

## Import Apple csv with selective columns

In [ ]:
csv_path = r'Resources/AppleStore.csv'
apple_columns = ['track_name', 'prime_genre', 'user_rating', 'rating_count_tot', 'price' ]
apple_df = pd.read_csv(csv_path, usecols=apple_columns)[apple_columns]

# Use google dataset naming conventions for the columns
# to avoid issues with inserting datat into pg, all column names are lowercase
apple_df.columns = [col.lower() for col in google_columns]

# make Category upper to align formatting
apple_df.category = apple_df.category.str.upper()

# convert rating, reviews, and price to numeric type
apple_df.rating = pd.to_numeric(apple_df.rating, errors='coerce')
apple_df.reviews = pd.to_numeric(apple_df.reviews, errors='coerce')
apple_df.price = pd.to_numeric(apple_df.price, errors='coerce')

# drop na rows
apple_df.dropna(inplace=True)

apple_df.head()

## Set up connection to database

In [ ]:
# connect to db
db = 'Apps_ETL_db'
port = '5432'
rds_connection_string = f"{username}:{password}@localhost:{port}/{db}"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Write Data to database

In [ ]:
google_df.to_sql(name='google_apps', con=engine, if_exists='append', index=False)
apple_df.to_sql(name='apple_apps', con=engine, if_exists='append', index=False)